In [3]:
using GLMakie, LinearAlgebra

include("boids.jl")

# TODO: repair reset
# TODO: Make boids not collided
# TODO: Make border limits for boids
# TODO: Try to make inifite run/stop animation
# TODO: Maybeeeeee try make .exe 

# Make boids
width = height = 200;
numBoids = 400
flock = [Boid(i, rand(-width:width, 3), randn(3).*3, randn(3).*2) for i=1:numBoids]
points = Observable([Point3f(boid.position) for boid in flock])

# Create figure
fig = Figure()

# create 3d axis
ax = Axis3(fig[1, 1], aspect = (1, 1, 1),
        title = "Flocking simulation")
        ax.perspectiveness = 0

# Set axis limits
limits!(ax, -2*width, 2*width, -2*width, 2*width, -2*width, 2*width)

# Create sliders layout
lsgrid = labelslidergrid!(fig,
    ["align", "separation", "cohesion"],
    Ref(LinRange(0.01:0.01:3));
    formats = [x -> "$(round(x, digits = 2))"],
    labelkw = Dict([(:textsize, 20)]),
    sliderkw = Dict([(:linewidth, 10)]),
    valuekw = Dict([(:textsize, 20)])
)

sl_sublayout = GridLayout(height=100)
fig[2, 1] = sl_sublayout
fig[2, 1] = lsgrid.layout

# Set initial parameters
set_close_to!(lsgrid.sliders[1], 0.5)
set_close_to!(lsgrid.sliders[2], 1.0)
set_close_to!(lsgrid.sliders[3], 0.6)

# Update parameters values
alignValue = lsgrid.sliders[1].value
separationValue = lsgrid.sliders[2].value
cohesionValue = lsgrid.sliders[3].value

# create button grid
fig[3, 1] = buttongrid = GridLayout(height=100)

labels = ["Run", "Step", "Restart"]
button_count = 3

# create buttons
buttons = buttongrid[3, 1:button_count] = [
    Button(fig,
        label = l, height = 30, width = 120, textsize = 30
    )
    for l in labels
]

# run button
on(buttons[1].clicks) do click
    frames = 1:100
    @async begin
    for i = frames
        for boid in flock
            fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
        end
        
        points[] = [Point3f(boid.position) for boid in flock]
        sleep(0.00001)
    end
    end
    # record(fig, "append_animation.mp4", frames;
    #         framerate = 30) do frame
    #     points[] += Point3f[(0., 0.01, 0.)]
    #     sleep(0.05)
    # end
end

# step button - the same as run
on(buttons[2].clicks) do click
    for boid in flock
        fly!(boid, flock, alignValue[], separationValue[], cohesionValue[])
    end
    
    points[] = [Point3f(boid.position) for boid in flock]
end

# reset button
on(buttons[3].clicks) do click
    delete!(flock)
    delete!(points)
    flock = [Boid(i, rand(-width:width, 3), randn(3).*3, randn(3)) for i=1:numBoids]
    # points[] = [Point3f(boid.position) for boid in flock]
    points = Observable([Point3f(boid.position) for boid in flock])
    points[] = [Point3f(boid.position) for boid in flock]
end

scatter!(ax, points, markersize=1000)

display(fig)

GLMakie.Screen(...)